<script type="text/javascript"
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML">
</script>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>

Jakub Musiał

# **AOD - lab2**

## **Exercise 3 - Police cars**

Knowing:
* The minimum and maximum number of police cars for each district and shift
* The minimum required number of police cars per district and shift

Determine the optimal allocation of police cars to each district and shift (minimum total number of police cars)

In [1]:
using JuMP
using GLPK

import JSON

#### **Data and utils**

In [2]:
# Data initialization
data_general = JSON.parse(read("data.json", String))
# Extraxt exercise 1 data from the general dictionary
data = data_general["ex3"]

Dict{String, Any} with 4 entries:
  "min_per_shift"    => Dict{String, Any}("s1"=>10, "s2"=>20, "s3"=>18)
  "max_no_cars"      => Dict{String, Any}("d3"=>Dict{String, Any}("s1"=>8, "s2"…
  "min_per_district" => Dict{String, Any}("d3"=>13, "d1"=>10, "d2"=>14)
  "min_no_cars"      => Dict{String, Any}("d3"=>Dict{String, Any}("s1"=>5, "s2"…

In [3]:
districts = sort(collect(keys(data["min_per_district"])))
shifts = sort(collect(keys(data["min_per_shift"])))
districts, shifts

(["d1", "d2", "d3"], ["s1", "s2", "s3"])

In [4]:
min_no(district::String, shift::String) = data["min_no_cars"][district][shift]
max_no(district::String, shift::String) = data["max_no_cars"][district][shift]
mpd(district::String) = data["min_per_district"][district]
mps(shift::String) = data["min_per_shift"][shift]

mps (generic function with 1 method)

#### **Build the model**

Notation:
* $D = \text{districts}$
* $S = \text{shifts}$

In [5]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

**Predictor variables:**

$x_{d, s}$ where $(d, s) \text{ } \epsilon \text{ } D \times S$ - number of police cars assigned to district $d$ during shift $s$

In [6]:
@variable(model, x[districts, shifts] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["d1", "d2", "d3"]
    Dimension 2, ["s1", "s2", "s3"]
And data, a 3×3 Matrix{VariableRef}:
 x[d1,s1]  x[d1,s2]  x[d1,s3]
 x[d2,s1]  x[d2,s2]  x[d2,s3]
 x[d3,s1]  x[d3,s2]  x[d3,s3]

**Constraints:** 
* Number of cars must be non-negative
  
  $(\forall{d \epsilon D})(\forall{s \epsilon S})(x_{d, s} >= 0)$

  
* Number of cars must be a value from a specified range
  
  $(\forall{d \epsilon D})(\forall{s \epsilon S})(\text{min\_no\_cars}(d, s) \leq x_{d, s} \leq \text{max\_no\_cars}(d, s))$

* The number of cars for each district and shift must be at least the minimum number assigned to that district or shift

  $
  \left\lbrace 
  \begin{array}{l}
  (\forall{d \epsilon D})(\sum_{s \epsilon S} x_{d, s} \geq \text{min\_per\_district}(d)) \newline
  (\forall{s \epsilon S})(\sum_{d \epsilon D} x_{d, s} \geq \text{min\_per\_shift}(s))
  \end{array}\right.
  $

In [7]:
@constraints(model, begin
    [d in districts, s in shifts], min_no(d, s) <= x[d, s] <= max_no(d, s)
    [d in districts], sum(x[d, :]) >= mpd(d)
    [s in shifts], sum(x[:, s]) >= mps(s)
end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.Interval{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["d1", "d2", "d3"]
    Dimension 2, ["s1", "s2", "s3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.Interval{Float64}}, ScalarShape}}:
 x[d1,s1] in [2.0, 3.0]  x[d1,s2] in [4.0, 7.0]   x[d1,s3] in [3.0, 5.0]
 x[d2,s1] in [3.0, 5.0]  x[d2,s2] in [6.0, 7.0]   x[d2,s3] in [5.0, 10.0]
 x[d3,s1] in [5.0, 8.0]  x[d3,s2] in [7.0, 12.0]  x[d3,s3] in [6.0, 10.0], 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["d1", "d2", "d3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIn

**Objective:** minimum number of cars which meets the given requirements

$min(\sum_{(d, s) \text{ } \epsilon \text{ } C \times S} x_{d, s})$

In [8]:
@objective(model, Min, sum(x[d, s] for d in districts, s in shifts))

x[d1,s1] + x[d1,s2] + x[d1,s3] + x[d2,s1] + x[d2,s2] + x[d2,s3] + x[d3,s1] + x[d3,s2] + x[d3,s3]

In [9]:
# Optimize the model
optimize!(model)
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 4.80000e+01
  Objective bound    : -Inf
  Dual objective value : 4.80000e+01

* Work counters
  Solve time (sec)   : 0.00000e+00


#### **Results**

In [11]:
for d in districts
    println("district: ", d)
    for s in shifts
        println("\t", s, " - ", value(x[d, s]))
    end
end

println("\ntotal no. cars: ", sum(value(x[d, s]) for d in districts, s in shifts))

district: d1
	s1 - 2.0
	s2 - 7.0
	s3 - 5.0
district: d2
	s1 - 3.0
	s2 - 6.0
	s3 - 7.0
district: d3
	s1 - 5.0
	s2 - 7.0
	s3 - 6.0

total no. cars: 48.0
